In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
from ast import literal_eval
from math import log10, log

from IPython.display import Markdown, display, HTML
from sklearn.metrics import recall_score
import pandas as pd

past_queries = ['criativo','Instagram','reprodução','real','hostil']

In [3]:
inv_index = pd.read_csv("../output/inverted_index.csv").set_index(['word'])\
            .drop('vivoÉ').drop('ônibusFoi') #remove nltk tokenization mistake
inv_index["doc_id:freq"] = inv_index["doc_id:freq"].apply(lambda x: literal_eval(x))
inv_index.head(5)

doc_id:freq
word                                                      
juíza                                     [(0, 2), (1, 1)]
federal  [(0, 1), (1, 1), (2, 1), (6, 2), (14, 1), (36,...
Ivani                                     [(0, 1), (1, 1)]
Silva    [(0, 3), (1, 1), (5, 1), (13, 2), (25, 1), (72...
Luz                             [(0, 3), (1, 1), (124, 1)]

In [4]:
def mim(**kwargs):
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    
    return n_ab/(n_a * n_b)

def dice_coef(**kwargs):
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return n_ab/(n_a + n_b)

def emim(**kwargs):
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]
    result = 0 if n_ab == 0 else \
             n_ab * log10(N * (n_ab / (n_a * n_b)))
    
    return result

def chi_square(**kwargs):
    N = 10e6 if 'N' not in kwargs else kwargs["N"]
    n_ab = kwargs["n_ab"]
    n_b = kwargs["n_b"]
    n_a = kwargs["n_a"]

    return pow((n_ab - (1/N) * n_a * n_b),2) / (n_a * n_b)

In [5]:
from bisect import insort_left


def n_most_related(n=10,**kwargs):
    metric = kwargs.pop("metric")
    index = kwargs.pop("df")
    A = kwargs.pop("A")    
    words = index.drop(A).reset_index()["word"].values
    rank = []
    
    for word in words:
        n_a = index.xs(A).values[0]
        n_b = index.xs(word).values[0]
        n_ab = [val1 for (val1,val2) in n_a if val1 \
            in (val1 for (val1,val2) in n_b)]
        kwargs["n_ab"] = len(n_ab)
        kwargs["n_a"] = len(n_a)
        kwargs["n_b"] = len(n_b)   
        insort_left(rank, (metric(**kwargs),word))
        rank = rank[-n:]
    
    return rank

In [6]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='Instagram',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'Instagram'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns=columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'Instagram'

***

* *EMIM* seemsto have the best results, as they have more to do with the term *Instagram*
    + The presence of the term 5 has to do with the preservation of dates in the inverted index

In [7]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='criativo',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'criativo'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'criativo'

***

* *EMIM* is the only algorithm whose result differs from the rest, being the less coherent at that.

In [8]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='reprodução',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'reprodução'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'reprodução'

***

* Chi-Squared seems to have the best results 

In [9]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='real',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'real'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'real'

***

* *EMIM* seems to have the best results, as in the more coherent with the term *real*

In [10]:
metrics = [mim,emim,chi_square,dice_coef]
columns=["MIM","EMIM","Chi-Squared","Dice"]
rows = []

for metric in metrics:
    top_10 = n_most_related(df=inv_index,A='hostil',
                            metric=metric,n=10)
    top_10 = list(reversed(top_10))
    top_10 = [i[1] for i in top_10]
    
    rows.append(top_10)
    
display(Markdown("***"))
display(Markdown("### Results for 'hostil'"))
display(HTML(pd.DataFrame(list(zip(*rows)), columns =columns).to_html(index=False)))
display(Markdown("***"))

***

### Results for 'hostil'

***

* *EMIM* seems to have the best results, as in the more coherent with the term *hostil*

> Overall **EMIM** seems to be the best choice, for its results are closer in terms of semantics to the original terms (The results are more coherent)

# Expading Queries

In [11]:
source = pd.read_csv("../output/results.csv")
source.head()

title  \
0  “A sociedade foi Rubens Paiva  não os facínora...   
1  Justiça suspende decisão que proibia Forças Ar...   
2  Governo Bolsonaro prega “negacionismo históric...   
3  Quando os pais de Gabo perceberam que tinham u...   
4  Rádios canadenses banem músicas de Michael Jac...   

                                            subtitle                 author  \
0  A decisão da juíza que proíbe as Forças Armada...                  F. M.   
1  Liminar havia sido concedida na sexta-feira a ...           Marina Rossi   
2  Marcos Napolitano  professor da USP  diz que o...       Regiane Oliveira   
3  Gustavo Tatis percorre o universo de García Má...    Jesús Ruiz Mantilla   
4  Quebec Cogeco Media toma a decisão após queixa...  Jaime Porras Ferreyra   

                  date  section  \
0  30/03/2019 00:11:08   Brasil   
1  30/03/2019 16:17:59   Brasil   
2  04/04/2019 22:37:48   Brasil   
3  07/03/2019 16:38:56  Cultura   
4  07/03/2019 16:12:37  Cultura   

                                                text  \
0  A juíza federal Ivani Silva da Luz  de Brasíli...   
1  Menos de 24 horas depois de a juíza federal Iv...   
2  Quando  determinou que  de 31 de março 1964  u...   
3  Quando  era pequeno   Luisa e  Gabriel se preo...   
4  Desde a manhã da última segunda-feira     e   ...   

                                                 url  
0  https://brasil.elpais.com/brasil/2019/03/26/po...  
1  https://brasil.elpais.com/brasil/2019/03/30/po...  
2  https://brasil.elpais.com/brasil/2019/04/05/po...  
3  https://brasil.elpais.com/brasil/2019/03/06/cu...  
4  https://brasil.elpais.com/brasil/2019/03/06/cu...

## Metrics

In [12]:
def recall_score(true_docs, predicted):
    true_p = [value for value in predicted if value in true_docs] 
    false_n = list(set(true_docs) - set(predicted))
    return len(true_p) / (len(true_p) + len(false_n))

def precision_score(true_docs, predicted):
    true_p = [value for value in predicted if value in true_docs] 
    false_p = list(set(predicted) - set(true_docs))
    return len(true_p) / (len(true_p) + len(false_p))

## Term At A Time Retrieval

In [13]:
def conjunctive_retrieval_by_term(index, query, k):
    """ Retrieve k matched documents sorted by score.

    Tranverses an inverted index term by term guided by a query 
    to retrieve k documents that contain all query terms 
    in decrescent order of score.
    
    :param pandas.core.frame.DataFrame index: inverted index.
    :param List[string] query: query terms
    :param int k: maximum result length .

    :return: list of documents sorted by score [(score,doc_id)] 

    :rtype: List[(int,int)]
    """
    
    try:
        import Queue as Q  # ver. < 3.0
    except ImportError:
        import queue as Q

    A = {}
    result = []
    R = Q.PriorityQueue()
    q = Q.PriorityQueue()
    L = index.loc[lambda df: df.word.isin(query)]

    for idx, row in enumerate(L.itertuples()):

        if idx == 0:
            for doc_id, freq in getattr(row, "_2"):
                A[doc_id] = freq        
        else:
            for doc_id, freq in getattr(row, "_2"):
                if doc_id not in A:
                    pass
                else:
                    A[doc_id] += freq        

    for doc_id, score in A.items():
        q.put(((-1) * score,doc_id))    

    i = 0
    while not q.empty() and i < k:
        pair = q.get()
        pair = ((-1) *pair[0],pair[1])
        result.append(pair)
        i += 1

    return result

## Instagram (Expanded Queries)

In [14]:
k = 10
term = 'Instagram'

In [15]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("***"))
display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))
display(Markdown("***"))

***

#### Result of *conjunctive retrieve by term* for 'Instagram'

***

In [16]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

title  \
85   Presidente na ficção  humorista é favorito em ...   
110  Bolsonaro se despede de Israel com promessas e...   
151                    Socialismo ‘millennial’ nos EUA   
229  Atos pró e contra ditadura militar terminam em...   
242            A fortuna blindada de Cristiano Ronaldo   

                                              subtitle  
85   Admirador declarado de Bolsonaro  o ator Volod...  
110  Presidente encerrou visita a aliado Netanyahu ...  
151  A temida palavra ‘socialista’ já não é mais ta...  
229  Grupos se atacaram com paus de bandeira  socos...  
242  Atacante criou um império empresarial em cuja ...

Above we have a list of the documents that mention the term *Instagram* directly

<br>

In [17]:
pd.set_option('max_colwidth', 200)
# pd.reset_option('^display.', silent=True)
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

title  \
64                                 O incerto futuro dos militares que desertaram na Venezuela   
2                           Governo Bolsonaro prega “negacionismo histórico” sobre a ditadura   
229                    Atos pró e contra ditadura militar terminam em confrontos em São Paulo   
165                                                          Bolsonaro manda festejar o crime   
137               Curto-circuito político faz da economia o calcanhar de Aquiles de Bolsonaro   
13                 Ricardo olha para a tela do celular e diz: “Eu estou falando comigo mesmo”   
46     Paulínia FC  o time da ‘Hollywood brasileira’ salvo pela venda de Fabinho ao Liverpool   
18                                               Bolsonaro (des)governa o Brasil pelo Twitter   
242                                                   A fortuna blindada de Cristiano Ronaldo   
114                       Esportes vivem paralisia sem planos detalhados e nomeações travadas   
85                 Presidente na ficção  humorista é favorito em disputa eleitoral na Ucrânia   
151                                                           Socialismo ‘millennial’ nos EUA   
24                 Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”   
187  O que aconteceria se os museus europeus tivessem que devolver a arte colonial espoliada?   
124                            A enorme (e inadiável) tarefa de regular o capitalismo digital   

                                                                                                                                                                                                    subtitle  \
64                                                       Mais de mil venezuelanos que desertaram há um mês aguardam em casas  hotéis e albergues sem esconder seu mal-estar quanto às condições em que vivem   
2    Marcos Napolitano  professor da USP  diz que o discurso do Governo Bolsonaro sobre o golpe de 64 está mais para negacionismo  pois “tem um ponto de partida ideológico  com objetivo de ocultar o pa...   
229                                                                                                                Grupos se atacaram com paus de bandeira  socos e chutes na avenida Paulista neste domingo   
165                                                              Ao determinar a comemoração do golpe militar de 1964  o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça   
137   Dificuldade de reverter o quadro é atribuída a vários fatores. Contas públicas no vermelho e contenção de gastos impedem que Governo impulsione a atividade econômica  como aconteceu em outras crises   
13                                                                 Em 'Ricardo e Vânia'  de Chico Felitti  os outros lados da vida do Fofão da Augusta  incluindo a história do seu grande amor. Leia trecho   
46                                               Clube de cidade que já foi polo cinematográfico do Brasil quita suas dívidas e quer voltar ao futebol profissional com dinheiro da transferência do jogador   
18                                                                                                            Ao tomar decisões pelo volume dos gritos nas redes sociais  o presidente corrompe a democracia   
242                                                                                            Atacante criou um império empresarial em cuja direção colocou sua família e um círculo muito íntimo de amigos   
114                           Rebaixada de ministério a secretaria  política esportiva de Bolsonaro espelha modelo do regime ditatorial  com militares na linha de frente  engessada pela falta de autonomia   
85                                    Admirador declarado de Bolsonaro  o ator Volodymyr Zelenskiy lidera as pesquisas para as eleições deste domingo explorando a insatisfação popular com a elite política   
151                A temida palavra ‘so

Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention Instagram directly but mention social networks for instance:
* 229, 18, 242,  85, 151

In [18]:
true_docs = [229, 18, 242,  85, 151] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

## Expanded Queries for term 'real'

In [19]:
k = 10
term = 'real'

In [20]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("***"))
display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))
display(Markdown("***"))

***

#### Result of *conjunctive retrieve by term* for 'real'

***

In [21]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

title  \
3                                             Quando os pais de Gabo perceberam que tinham um filho mentiroso   
4                                           Rádios canadenses banem músicas de Michael Jackson da programação   
11                                                                 As imagens questionadoras de Harun Farocki   
12                         O manuscrito de ‘Cem anos de solidão’ que García Márquez acreditava ter se perdido   
15                                                                                      A tragédia da Ucrânia   
16    Por que o papa Francisco ordenou a abertura dos arquivos secretos do Vaticano sobre o Holocausto judeu?   
17                                                                                      Um tuíte muito vulgar   
18                                                               Bolsonaro (des)governa o Brasil pelo Twitter   
19                                                                                          Igualdade radical   
21                                                  Taurus  ascensão de uma fabricante de armas com Bolsonaro   
22                                               Davos esvaziada recebe Bolsonaro como astro da nova política   
25                                                          O breve discurso de Bolsonaro decepciona em Davos   
27                                         Puxado por Brasil e Espanha  lucro do Santander cresce 18% em 2018   
28                                                     Europa busca se reconciliar com seu violento século XX   
29                                                         Os entregadores de comida são reféns do seu ‘like’   
30                                                                             Matando como em ‘Call of Duty’   
35                            Finlândia conclui experiência de renda básica universal com resultados ambíguos   
36                               Os percalços dos planos A e B de Paulo Guedes para controlar o gasto público   
40                                                                    Turim se apaixona por Cristiano Ronaldo   
41                             Após acenar com embaixada  Bolsonaro anuncia escritório comercial em Jerusalém   
42                       “Mostre o que as loucas podem fazer”: o anúncio da Nike sobre as mulheres no esporte   
45                                                                           O trágico destino de Tom Ballard   
54                 Eder Militão  o zagueiro mais caro da história do Real Madrid e primeiro reforço de Zidane   
58                                                                    Greve de mulheres na redação do EL PAÍS   
59                              A mecenas que levou a obra de 16 brasileiros ao Museu Reina Sofía  na Espanha   
62                     “Chamar-me de traidor foi uma estratégia muito bem montada para minar minha liderança”   
63                                    De surpresa  Trump revoga as últimas sanções impostas à Coreia do Norte   
68         Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”   
75                                                                   Crianças condenadas ao fogo e ao descaso   
76                                               Trump reconhece soberania de Israel sobre as Colinas de Golã   
..                                                                                                        ...   
194                    ‘Tears in Heaven’: a tragédia que inspirou uma das canções mais comoventes da história   
195                                                         Tudo funciona bem em Mick Jagger  menos o coração   
196      O que a fama fez com Susan Boyle  a mulher que revolucionou a música  a televisão e as redes sociais   
197                                                          Todos os filmes de Tim Burton  do pior ao melhor   
200                                 

Above we have a list of the documents that mention the term *real* directly

<br>

In [22]:
pd.set_option('max_colwidth', 200)
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

title  \
129              A universidade que quer oferecer uma educação do nível de Harvard pela metade do preço   
68   Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”   
165                                                                    Bolsonaro manda festejar o crime   
137                         Curto-circuito político faz da economia o calcanhar de Aquiles de Bolsonaro   
75                                                             Crianças condenadas ao fogo e ao descaso   
172                             Não temos tempo a perder. A hora de unir as forças democráticas é agora   
173                                                                Os sete tiros de fuzil do presidente   
206                    Dois assessores de Bolsonaro doaram mais de 100.000 reais a campanhas da família   
17                                                                                Um tuíte muito vulgar   
18                                                         Bolsonaro (des)governa o Brasil pelo Twitter   
114                                 Esportes vivem paralisia sem planos detalhados e nomeações travadas   
25                                                    O breve discurso de Bolsonaro decepciona em Davos   
85                           Presidente na ficção  humorista é favorito em disputa eleitoral na Ucrânia   
150                                                              Quem mandou matar Marielle? E por quê?   
215                                                   Um Brasil de dedo em riste às portas do Mackenzie   
24                           Boris Fausto e o golpe de 64: “É impossível ir contra fatos estabelecidos”   
21                                            Taurus  ascensão de uma fabricante de armas com Bolsonaro   
62               “Chamar-me de traidor foi uma estratégia muito bem montada para minar minha liderança”   

                                                                                                                                                                                                    subtitle  \
129                                                                Chama-se Minerva  e seus alunos estudam em sete cidades espalhadas pelo mundo  provocando uma revolução no panorama universitário dos EUA   
68                  Em entrevista ao EL PAÍS  o ideólogo da extrema direita diz que o vice-primeiro-ministro italiano e o premiê húngaro Viktor Orbán são os políticos mais importantes da Europa atualmente   
165                                                              Ao determinar a comemoração do golpe militar de 1964  o antipresidente busca manter o ódio ativo e barrar qualquer possibilidade de justiça   
137   Dificuldade de reverter o quadro é atribuída a vários fatores. Contas públicas no vermelho e contenção de gastos impedem que Governo impulsione a atividade econômica  como aconteceu em outras crises   
75                                       Creche em Janaúba  Museu Nacional e os garotos do Ninho. Retratos de uma nação que queima o passado com a mesma passividade que destrói suas perspectivas de futuro   
172                                                               Essa articulação requererá deixar de lado diferenças e firmar um compromisso pela defesa do interesse da sociedade brasileira como um todo   
173                                                               A guerra  a das armas e das ideologias  é a grama que melhor parece crescer nesse jardim sombrio do capitão reformado caçador de conflitos   
206                                         Levantamento da Pública mostra que outros cinco assessores prestaram serviços de campanha enquanto estavam contratados pelo atual presidente ou seus três filhos   
17                                                                                                       Bolsonaro lança cortina de fumaça nas redes sociais em detrimento da imagem institucion

Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'real' directly (e.g. talk about reality or brazilian currency):
* 172, 68, 75, 173, 206, 17, 18, 114, 25, 85, 150, 215, 24, 62

In [23]:
true_docs = [172, 68, 75, 173, 206, 17, 18, 114, 25, 85, 150, 215, 24, 62] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

## Expanded Queries for term 'reprodução'

In [24]:
k = 10
term = 'reprodução'

In [25]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("***"))
display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))
display(Markdown("***"))

***

#### Result of *conjunctive retrieve by term* for 'reprodução'

***

In [26]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

title  \
88                                      Bruno Latour: “O sentimento de perder o mundo  agora  é coletivo”   
123                               França impulsiona o acesso juvenil à cultura com um cheque de 500 euros   
140                                                 Spotify lança publicidade apenas para pessoas felizes   
160                                         Violência obstétrica  uma forma de desumanização das mulheres   
196  O que a fama fez com Susan Boyle  a mulher que revolucionou a música  a televisão e as redes sociais   

                                                                                                                                                                                            subtitle  
88                                                                                        Em seu último livro  o influente pensador francês descreve um planeta onde a mudança climática altera tudo  
123                Governo de Macron testa aplicativo com geolocalização para celular que convida aqueles que completam 18 anos a gastar em cinemas  teatros  livros  cursos ou assinaturas ‘online’  
140                  A plataforma de música em ‘streaming’ estreia com a Renault sua primeira campanha de ‘happy targeting’  que só mostra os anúncios a quem ouve listas de reprodução mais alegres  
160                                                               O fenômeno é muito mais comum do que a novidade da palavra parece sugerir: são muitas as mulheres que ignoram ter sofrido com isso  
196  Há dez anos  uma total desconhecida tornou-se  da noite para o dia  a mulher mais famosa do mundo. É assim que ela vive hoje  depois de conhecer a glória e a riqueza  mas também a obscuridade

Above we have a list of the documents that mention the term *reprodução* directly

<br>

In [27]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

title  \
5                          Rosângela  uma das crianças sequestradas por famílias de militares na ditadura   
8                                                                          O lado mais sombrio de Dickens   
140                                                 Spotify lança publicidade apenas para pessoas felizes   
13                             Ricardo olha para a tela do celular e diz: “Eu estou falando comigo mesmo”   
18                                                           Bolsonaro (des)governa o Brasil pelo Twitter   
150                                                                Quem mandou matar Marielle? E por quê?   
32                                                                     A ética da solidariedade feminista   
160                                         Violência obstétrica  uma forma de desumanização das mulheres   
165                                                                      Bolsonaro manda festejar o crime   
182                             Gal Costa: “É um disco alegre para tempos sombrios  no Brasil e no mundo”   
62                 “Chamar-me de traidor foi uma estratégia muito bem montada para minar minha liderança”   
194                ‘Tears in Heaven’: a tragédia que inspirou uma das canções mais comoventes da história   
196  O que a fama fez com Susan Boyle  a mulher que revolucionou a música  a televisão e as redes sociais   
68     Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”   
197                                                      Todos os filmes de Tim Burton  do pior ao melhor   
75                                                               Crianças condenadas ao fogo e ao descaso   
210                            “Refletir sobre nosso comportamento online é mais importante do que votar”   
88                                      Bruno Latour: “O sentimento de perder o mundo  agora  é coletivo”   
238                              Saúde mental dos estudantes  mais um desafio para as escolas brasileiras   
111      Idoso é preso por ajudar esposa a fazer eutanásia após 30 anos de luta contra esclerose múltipla   

                                                                                                                                                                                                    subtitle  \
5                                                                                              “Roubaram minha identidade”  diz Rosângela Paraná  cuja certidão de nascimento foi forjada pelo avô sargento.   
8                                                                                                        Escritor e jornalista tentou internar sua mulher em um manicômio para poder se relacionar com atriz   
140                          A plataforma de música em ‘streaming’ estreia com a Renault sua primeira campanha de ‘happy targeting’  que só mostra os anúncios a quem ouve listas de reprodução mais alegres   
13                                                                 Em 'Ricardo e Vânia'  de Chico Felitti  os outros lados da vida do Fofão da Augusta  incluindo a história do seu grande amor. Leia trecho   
18                                                                                                            Ao tomar decisões pelo volume dos gritos nas redes sociais  o presidente corrompe a democracia   
150                                                                                         Bolsonaro  que governa o Brasil pela administração do ódio  deveria ser o maior interessado em desvendar o crime   
32                                                                  Se é preciso ouvir antes de falar  é também nosso dever falar com responsabilidade sobre as injustiças que assolam as mulheres e meninas   
160                                                                       O fenômeno é muito mais comum do que a novidade da palavra parece sugeri

Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'reprodução' directly (e.g. talk about copying):
* 5, 8, 150, 160, 196, 68, 197, 88

In [28]:
true_docs = [5, 8, 150, 160, 196, 68, 197, 88] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

## Expanded Queries for term 'criativo'

In [29]:
k = 10
term = 'criativo'

In [30]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("***"))
display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))
display(Markdown("***"))

***

#### Result of *conjunctive retrieve by term* for 'criativo'

***

In [31]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

title  \
35   Finlândia conclui experiência de renda básica universal com resultados ambíguos   
150                                           Quem mandou matar Marielle? E por quê?   

                                                                                                                                                                           subtitle  
35   Durante dois anos  2.000 pessoas desempregadas receberam 2.370 reais por mês. Saúde  autoestima e otimismo dos beneficiários melhoraram  mas situação profissional pouco mudou  
150                                                                Bolsonaro  que governa o Brasil pela administração do ódio  deveria ser o maior interessado em desvendar o crime

Above we have a list of the documents that mention the term *criativo* directly

<br>

In [32]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

title  \
225                                                                  Neto de Lula não morreu de meningite   
162                                                                                 A era dos reptilianos   
35                        Finlândia conclui experiência de renda básica universal com resultados ambíguos   
68     Steve Bannon: “Bolsonaro e Salvini são os melhores representantes do movimento nacional-populista”   
196  O que a fama fez com Susan Boyle  a mulher que revolucionou a música  a televisão e as redes sociais   
8                                                                          O lado mais sombrio de Dickens   
238                              Saúde mental dos estudantes  mais um desafio para as escolas brasileiras   
79                                 Dois sobreviventes do massacre de Parkland se suicidam na mesma semana   
210                            “Refletir sobre nosso comportamento online é mais importante do que votar”   
150                                                                Quem mandou matar Marielle? E por quê?   
22                                           Davos esvaziada recebe Bolsonaro como astro da nova política   
216                Operando com 10% do orçamento  Funai abandona postos e coordenações em áreas indígenas   
25                                                      O breve discurso de Bolsonaro decepciona em Davos   
62                 “Chamar-me de traidor foi uma estratégia muito bem montada para minar minha liderança”   

                                                                                                                                                                                                    subtitle  \
225                        Um mês depois da morte  laudo de exames realizados pelo Instituto Adolfo Lutz descartam todos os tipos de meningite como causa do óbito de Arthur Araújo Lula da Silva  de 7 anos   
162                                                                                  As teorias da conspiração pressupõem que os dois lados de uma disputa científica ou social devem ter a mesma veracidade   
35                            Durante dois anos  2.000 pessoas desempregadas receberam 2.370 reais por mês. Saúde  autoestima e otimismo dos beneficiários melhoraram  mas situação profissional pouco mudou   
68                  Em entrevista ao EL PAÍS  o ideólogo da extrema direita diz que o vice-primeiro-ministro italiano e o premiê húngaro Viktor Orbán são os políticos mais importantes da Europa atualmente   
196          Há dez anos  uma total desconhecida tornou-se  da noite para o dia  a mulher mais famosa do mundo. É assim que ela vive hoje  depois de conhecer a glória e a riqueza  mas também a obscuridade   
8                                                                                                        Escritor e jornalista tentou internar sua mulher em um manicômio para poder se relacionar com atriz   
238               Massacre de Suzano reabre debate sobre papel das instituições de ensino em identificar transtornos psiquiátricos. 80% dos alunos da rede pública com algum problema não recebem tratamento   
79                                                                                                                 Comunidade escolar da Flórida  nos EUA  pede mais recursos para programas de saúde mental   
210                                                                                                                            Pesquisador inglês Jamie Bartlett chama atenção para a cidadania na Internet:   
150                                                                                         Bolsonaro  que governa o Brasil pela administração do ódio  deveria ser o maior interessado em desvendar o crime   
22                                        Fórum Econômico Mundial estende seu tapete vermelho neste ano ao recém-empossado presidente do Brasil  numa edição com reduzida presen

Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'criativo' directly (e.g. talk about creativity):
* 35, 210, 150

In [33]:
true_docs = [35, 210, 150] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))

In [34]:
k = 10
term = 'hostil'

In [35]:
top_10 = n_most_related(df=inv_index,A=term,metric=emim,n=10)
top_10 = list(reversed(top_10))
top_10 = [i[1] for i in top_10]
top_10 = [term] + top_10
conj_queries = [str(top_10[0:4]),str(top_10[0:6]),str(top_10)]

df_queries = pd.DataFrame(conj_queries, columns=["query"])
df_queries["query"] = df_queries["query"].apply(lambda x: list(literal_eval(x)))
df_queries["top_n_plus"] = df_queries["query"].apply(lambda w : len(w) -1)
df_queries["documents"] = df_queries["query"]\
                                .apply(lambda w: conjunctive_retrieval_by_term(inv_index.reset_index(), w, k))
df_queries["documents"] = df_queries["documents"].apply(lambda d: list(p[1] for p in d))

display(Markdown("***"))
display(Markdown("#### Result of *conjunctive retrieve by term* for '{}'".format(term)))
display(HTML(df_queries.to_html(index=False)))
display(Markdown("***"))

***

#### Result of *conjunctive retrieve by term* for 'hostil'

***

In [36]:
direct_docs = list(source.loc[source.apply(lambda x: term in x["text"], axis=1)].index.values)
source.iloc[direct_docs,:].drop(['date','text','section','url','author'],axis=1)

title  \
5            Rosângela  uma das crianças sequestradas por famílias de militares na ditadura   
91                                  O brasileiro insensato que rebate racismo com xenofobia   
94          No Chile  a política repudia a figura de Pinochet e matiza seu legado econômico   
216  Operando com 10% do orçamento  Funai abandona postos e coordenações em áreas indígenas   
245             Israel ataca Gaza e o Hamas lança foguetes apesar do anúncio de cessar-fogo   

                                                                                                                                                                                                    subtitle  
5                                                                                              “Roubaram minha identidade”  diz Rosângela Paraná  cuja certidão de nascimento foi forjada pelo avô sargento.  
91                   Jogador Serginho foi vítima de ofensas racistas e abandonou o campo na Bolívia. Por aqui  alguns torcedores preferem ofender o país vizinho a refletir sobre seus próprios preconceitos  
94   Piñera já fez vários gestos de rejeição ao passado do regime que matou mais de 3.000 chilenos. O legado econômico da ditadura também é contestado pela sociedade que cobra Educação e Previdência co...  
216                                                         Em zonas de conflito  há coordenações que funcionam dentro de carros e funcionários que sofrem de esgotamento físico e mental por atuar sozinhos  
245                                                                                                                 Exército concentra tropas e limita as atividades civis na fronteira do enclave palestino

Above we have a list of the documents that mention the term *hostil* directly

<br>

In [37]:
documents = list(set(df_queries["documents"].sum()))
source.iloc[documents,:].drop(['author', 'date','text','section'], axis=1)

title  \
5                      Rosângela  uma das crianças sequestradas por famílias de militares na ditadura   
39           Maior elétrica dos EUA declara falência por responsabilidade nos incêndios na Califórnia   
168              Na Argentina  falar da ditadura e dos militares que a conduziram é motivo de desonra   
8                                                                      O lado mais sombrio de Dickens   
75                                                           Crianças condenadas ao fogo e ao descaso   
44                                            Vinicius Junior decola no Real Madrid e chega à seleção   
203  A queda de outro ex-presidente do Brasil  a nova fatura da Lava Jato à velha política brasileira   
243                                               Nanga Parbat  a obsessão com a ‘montanha assassina’   
149        Grupos que sustentam Maduro serão principal desafio a qualquer futuro governo na Venezuela   
216            Operando com 10% do orçamento  Funai abandona postos e coordenações em áreas indígenas   
94                    No Chile  a política repudia a figura de Pinochet e matiza seu legado econômico   
127                                       Tragédia em Brumadinho derruba grau de investimento da Vale   

                                                                                                                                                                                                    subtitle  \
5                                                                                              “Roubaram minha identidade”  diz Rosângela Paraná  cuja certidão de nascimento foi forjada pelo avô sargento.   
39                                                                                                            Pacific Gas & Electric enfrenta centenas de processos por negligência em incêndios com mortos.   
168                                                                                                                     País teve um dos regimes mais sanguinários do continente  com ao menos 30.000 mortos   
8                                                                                                        Escritor e jornalista tentou internar sua mulher em um manicômio para poder se relacionar com atriz   
75                                       Creche em Janaúba  Museu Nacional e os garotos do Ninho. Retratos de uma nação que queima o passado com a mesma passividade que destrói suas perspectivas de futuro   
44                                                                                          Agora referência do ataque merengue  o jogador de 18 anos é o mais novo a ser convocado para o Brasil desde 2011   
203  Ninguém podia imaginar em 2014 que a operação revelaria o maior escândalo de corrupção da história do Brasil e seria o estopim de mudanças profundas que se materializaram em um presidente de ultra...   
243                                                                  A recente morte de dois alpinistas no topo mais letal depois do Annapurna acentua a angustiante luta com este cume há mais de um século   
149                                                                          É erro grave presumir que queda de Maduro e promoção de eleições livres seriam garantia para resolver problemas do país vizinho   
216                                                         Em zonas de conflito  há coordenações que funcionam dentro de carros e funcionários que sofrem de esgotamento físico e mental por atuar sozinhos   
94   Piñera já fez vários gestos de rejeição ao passado do regime que matou mais de 3.000 chilenos. O legado econômico da ditadura também é contestado pela sociedade que cobra Educação e Previdência co...   
127                                                                                                                    Agência de risco Moody's rebaixou  em escala global  ratings da mineradora brasileira   

                         

Taking a closer look at each document, the following ones can be classified as relevant even if they don't mention the word 'hostil' directly (e.g. talk about violence/hostility):
* 5, 168, 8, 75, 149, 216, 94

In [38]:
true_docs = [5, 168, 8, 75, 149, 216, 94] + direct_docs
true_docs = list(set(true_docs))

df_queries["precision"] = df_queries["documents"].apply(lambda docs: precision_score(true_docs, docs))
df_queries["recall"] = df_queries["documents"].apply(lambda docs: recall_score(true_docs, docs))

display(HTML(df_queries.to_html(index=False)))